# Infection forecast and Risk Index - Italian Regions 

Erika Agostinelli

## Part I 
Dedicated to the Confirmed cases and Mortality forecast of all italian regions 

## Part II 
Dedicated to the creation of the Risk Index 


### Libraries

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
try:
    import filterpy
except ImportError:
    !pip install filterpy
from filterpy.kalman import KalmanFilter, EnsembleKalmanFilter, UnscentedKalmanFilter, MerweScaledSigmaPoints
from filterpy.common import Q_discrete_white_noise, Saver
import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)

     |################################| 184kB 6.2MB/s eta 0:00:01
  Created wheel for filterpy: filename=filterpy-1.4.5-cp36-none-any.whl size=110452 sha256=82f403ae50ed4325b21bf50225cdb773d9798498f5e6b7a3f292590f705052a4
  Stored in directory: /home/wsuser/.cache/pip/wheels/c3/0c/dd/e92392c3f38a41371602d99fc77d6c1d42aadbf0c6afccdd02
Successfully built filterpy


In [53]:
!pip install ipywidgets

In [54]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

## Part I
### Kalman filters

In [2]:
def kalman_predictor(initial_state, kf_p, kf_r, kf_q, kf_a):
    """
    We model Covid development as a dynamical system composed of 3 components:
    - measurement (observable) = case count,
    - speed (latent) = growth rate (cases per day)
    - acceleration (latent) = growth acceleration (cases per day^2)    
    - used params of kf_p=0, kf_r=10, kf_q=20
    """
    # day is our observation interval
    dt = 1
    # transition matrix (x:measurement, v:growth rate, a:growth acceleration)
    F = np.array([[1, dt, 0.5*(dt**2)], # x_new           = x_old + v*dt + 1/2*a*dt^2
                    [0, 1, dt],           # d(v_new) / dt   = v     + a*dt + 0
                    [0, 0, 1]])           # d(x_new) / dt^2 = 0     + 0    + a
    '''
    F = np.array([[1, dt, (dt**2)/2, (dt**3)/6], # x_new = x_old + v*dt + 1/2*a*dt^2 + 1/6*j*dt^3
                  [0, 1, dt, (dt**2)/2],         # d(x_new) / dt = v + a*dt + 1/2*j*dt^2
                  [0, 0, 1, dt],                 # d(v_new) / dt = a + jt
                  [0, 0, 0, 1]])                 # d(a_new) / dt = j
    '''
    # define a linear KF with position, velocity, acceleration parameters
    dim_x = F.shape[0]
    kf = KalmanFilter(dim_x=dim_x, dim_z=1)
    kf.F = F
    # state vector: initial position, velocity, acceleration
    kf.x = np.zeros(dim_x)
    kf.x[0] = initial_state
    # measuremnet matrix: can only directly measure case counts, not velocity & acceleration
    kf.H = np.zeros((1, dim_x))
    kf.H[0][0] = 1
    # covariance matrix
    kf.P *= kf_p
    # measurement noise
    kf.R = kf_r
    # process noise
    kf.Q = Q_discrete_white_noise(dim=dim_x, dt=1, var=kf_q)
    # fading factor
    kf.alpha = kf_a
    return kf

def ensemble_kalman_predictor(initial_state, kf_p, kf_r, kf_q, kf_n):
    """
    We model Covid development as a dynamical system composed of 3 components:
    - measurement (observable) = case count,
    - speed (latent) = growth rate (cases per day)
    - acceleration (latent) = growth acceleration (cases per day^2)
    """
    # day is our observation interval
    dt = 1
    # transition matrix (x:measurement, v:growth rate, a:growth acceleration)
    F = np.array([[1, dt, (dt**2)/2], # x_new           = x_old + v*dt + 1/2*a*dt^2
                  [0, 1, dt],           # d(x_new) / dt   = v     + a*dt + 0
                  [0, 0, 1]])           # d(x_new) / dt^2 = 0     + 0    + a
    ''' also consider jerk
    F = np.array([[1, dt, (dt**2)/2, (dt**3)/6], # x_new = x_old + v*dt + 1/2*a*dt^2 + 1/6*j*dt^3
                  [0, 1, dt, (dt**2)/2],         # d(x_new) / dt = v + a*dt + 1/2*j*dt^2
                  [0, 0, 1, dt],                 # d(v_new) / dt = a + jt
                  [0, 0, 0, 1]])                 # d(a_new) / dt = j
    '''
    # state vector: initial position, velocity, acceleration
    X0 = np.zeros(F.shape[0])
    X0[0] = initial_state
    # transition function
    Fx = lambda x, dt: np.dot(F, x)
    # measuremnet function
    Hx = lambda x: np.array([x[0]])
    # covariance matrix
    P = np.eye(F.shape[0]) * kf_p
    # measurement noise
    R = kf_r
    # process noise
    Q = Q_discrete_white_noise(dim=F.shape[0], dt=1, var=kf_q)
    # let's make it
    kf = EnsembleKalmanFilter(x=X0, P=P, dim_z=1, dt=1, N=kf_n, hx=Hx, fx=Fx)
    kf.R = R
    kf.Q = Q
    return kf

### Forecast

In [3]:
def kalman_forecast(series, days, kf_type, params):
    """
    Forecast based on history data.
    
    Input
    -----
    series: Pandas Series object with dates being index in ascending order.
    days: Prediction window length
    kf_type: linear, unscented, ensemble
    kf_*: Parameters for Kalman filter. Default values work reasonably well on several countries.
    
    Output
    ------
    Pandas DataFrame object with the following columns
    pred_raw: Raw prediction
    pred: Final prediction (with smoothing, etc.)
    ci_*: Lower and upper bounds of CI
    """
    if days <= 0:
        raise ValueError
    dates = series.index
    if kf_type == 'linear':
        if params is None:
            params = {'kf_p':1, 'kf_r':4, 'kf_q':0.1, 'kf_a':1}
        if params['kf_a'] < 1:
            raise ValueError
        kf = kalman_predictor(series[dates[0]], params['kf_p'], params['kf_r'], params['kf_q'], params['kf_a'])
    elif kf_type == 'ensemble':
        if params is None:
            params = {'kf_p':100, 'kf_r':1000, 'kf_q':0.1, 'kf_n':1000}
        kf = ensemble_kalman_predictor(series[dates[0]], params['kf_p'], params['kf_r'], params['kf_q'], params['kf_n'])
    else:
        raise NotImplementedError
    
    # fit model
    for measurement in series:
        kf.predict()
        kf.update([measurement])
    
    # start forecasting, starting from the last observation date
    last_date = dt.datetime.strptime(dates[-1], '%m-%d')
    predictions = [series[-1]]
    pred_dates = [dates[-1]]
    ci_bounds = [0]
    for day in range(days):
        future_date = (last_date + dt.timedelta(days=day+1)).strftime('%m-%d')
        pred_dates.append(future_date)
        kf.predict()
        predictions.append(kf.x[0])
        ci_bounds.append(kf_ci_bound(kf))
    
    # smoothen and add confidence intervals
    predictions = np.array(predictions)
    predictions[np.where(predictions < 0)[0]] = 0
    #smooth_buffer = list(series[dates[len(dates)-days+1:]])
    #predictions_smooth = smoother(smooth_buffer + predictions, days)[days-1:]
    predictions_smooth = smoother(predictions, days)
    ci_bounds = np.array(ci_bounds)
    ci_upper = predictions_smooth + ci_bounds
    ci_lower = predictions_smooth - ci_bounds
    ci_lower[np.where(ci_lower < 0)[0]] = 0
    
    df_pred = pd.DataFrame({'pred_raw':predictions, 'pred':predictions_smooth,
                          'ci_lower':ci_lower, 'ci_upper':ci_upper}, index=pred_dates)
    return df_pred

def kf_ci_bound(kf):
    """
    Compute 95% confidence interval from KF's positive semi-definite covariance matrix
    
    returns a positive single-sided boundary (half) of the interval
    -> CI = kf.x[0] +- kf_ci_bound(kf)
    """
    return 1.96 * (np.diag(kf.P)[0])**0.5

def smoother(x, winsize, method='slide'):
    if method == 'slide':
        x_smooth = []
        for i in range(len(x)):
            x_smooth.append(np.mean(x[max(0, i-winsize+1):i+1]))
    elif method == 'slide_recurse':
        x_smooth = predictions.copy()
        for i in range(len(x)):
            x_smooth[i] = np.mean(x_smooth[max(0, i-winsize):i+1])
    else:
        raise NotImplementedError
    assert(len(x) == len(x_smooth))
    return np.array(x_smooth)

def get_stats(observations, predictions):
    r2 = r2_score(observations, predictions)
    mae = mean_absolute_error(observations, predictions)
    rmse = mean_squared_error(observations, predictions) ** 0.5
    return r2, mae, rmse

def rescale(df_x):
    x = df_x.copy()
    x -= x.min()
    x /= x.max()
    return x

### Performance validation

In [4]:
def kalman_test(series, winsize, kf_type, params=None):
    """
    To test the performance compared with true values along all data points
    
    Input
    -----
    series: Pandas Series object with dates being index in ascending order.
    winsize: Prediction window size in number of days.
    kf_type: linear, unscented, ensemble
    kf_*: factors for Kalman filter. Default values work reasonably well.
          For long-term prediction, usually increasing the fading factor (kf_a) helps.
    
    Output
    ------
    Pandas DataFrame object with the following columns
    pred_raw: Raw prediction
    pred: Final prediction (with smoothing, etc.)
    obs: Ground-truth values
    history: recursive prediction history at each time point (for debugging purpose)
    """
    if winsize <= 0:
        raise ValueError
    observations = []
    predictions = []
    pred_dates = []
    history = []
    ci_bounds = []
    dates = series.index.to_numpy()
    if kf_type == 'linear':
        if params is None:
            params = {'kf_p':1, 'kf_r':4, 'kf_q':0.1, 'kf_a':1}
        if params['kf_a'] < 1:
            raise ValueError
        kf = kalman_predictor(series[dates[0]], params['kf_p'], params['kf_r'], params['kf_q'], params['kf_a'])
    elif kf_type == 'ensemble':
        if params is None:
            params = {'kf_p':100, 'kf_r':1000, 'kf_q':0.1, 'kf_n':1000}
        kf = ensemble_kalman_predictor(series[dates[0]], params['kf_p'], params['kf_r'], params['kf_q'], params['kf_n'])
    else:
        raise NotImplementedError(kf_type)
    
    for i in range(dates.shape[0]-winsize):
        # save the current state of the model
        saver = Saver(kf, skip_callable=True, save_current=True)
        
        # recursive prediction
        history_window = [kf.x[0]]
        date_window = [dates[i]]
        for day in range(winsize):
            kf.predict()
            history_window.append(kf.x[0])
            date_window.append(dates[i+day+1])
        history.append(pd.DataFrame({'pred':history_window}, index=date_window))
        pred_date = dates[i+winsize]
        pred_dates.append(pred_date)
        prediction = kf.x[0]
        predictions.append(prediction)
        observation = series[pred_date]
        observations.append(observation)
        ci_bounds.append(kf_ci_bound(kf))
        
        # restore model states and update to next day
        for attr in saver.keys:
            try:
                setattr(kf, attr, getattr(saver, attr)[-1])
            except AttributeError: # property decoration causes problem
                #print('.%s skip' % attr)
                continue
        kf.predict()
        kf.update([series[dates[i+1]]])
    
    # smoothen output
    predictions_smooth = smoother(predictions, winsize)
    predictions_smooth[predictions_smooth < 0] = 0
    ci_bounds = np.array(ci_bounds)
    ci_lower = predictions_smooth - ci_bounds
    ci_upper = predictions_smooth + ci_bounds
    ci_lower[np.where(ci_lower < 0)[0]] = 0
    df_pred = pd.DataFrame({'pred_raw':predictions[winsize:], 'pred':predictions_smooth[winsize:],
                          'obs':observations[winsize:], 'history':history[winsize:],
                          'ci_lower':ci_lower[winsize:], 'ci_upper':ci_upper[winsize:]}, index=pred_dates[winsize:])
    return df_pred

### Predict & Plot

In [5]:
def test_plot(series, winsize, kf_type, params=None, title='', y_lims=None):
    """
    Make a prediction and plot at once for convenience.
    
    """
    PLOT_RAW = True
    SHOW_TRAJECTORY = True
    
    result = kalman_test(series, winsize, kf_type, params)
    forecast = kalman_forecast(series, winsize, kf_type, params)
    r2, mae, rmse = get_stats(result.obs, result.pred)
    #r2, mae, rmse = get_stats(result.obs, result.pred_raw)
    plt.rcParams.update({'figure.max_open_warning': 0})
    plt.figure(figsize=[13,5])
    plt.plot(series.index, series, 'o', color='g', linewidth=3)
    plt.plot(result.index, result.pred, '-', color=(1,0,0,0.8), linewidth=3)
    if PLOT_RAW:
        plt.plot(result.index, result.pred_raw, 'o', color=(1,0,0,0.3), linewidth=3)
        plt.plot(forecast.index, forecast.pred_raw, 'o', color=(1,0,0,0.3), linewidth=3)
    plt.fill_between(result.index, result.ci_lower, result.ci_upper, color=(1,0,0,0.1))
    plt.plot(forecast.index, forecast.pred, color='r', linewidth=3)
    plt.fill_between(forecast.index, forecast.ci_lower, forecast.ci_upper, color=(1,0,0,0.1))
    if SHOW_TRAJECTORY:
        for df_h in result.history:
            plt.plot(df_h.index, df_h.pred, '-', color=(0,0,1,0.2))
    if y_lims:
        plt.ylim(y_lims)
    plt.xticks(rotation=45, fontsize='small')
    plt.title(r'%s ($r^2$:%.3f, mae:%d, rmse:%d=%.1f%% of global max %d)' % (title, r2, mae, rmse, rmse * 100 / series.max(), series.max()))
    plt.legend(['True', 'Prediction'])
    plt.grid(True, 'both')
    plt.tight_layout()

In [6]:
def forecast_plot(series, winsize, kf_type, params=None, title='', y_lims=None):
    """
    Make a prediction and plot at once for convenience.    
    """
    PLOT_RAW = True
    
    forecast = kalman_forecast(series, winsize, kf_type, params)
    plt.rcParams.update({'figure.max_open_warning': 0})
    plt.figure(figsize=[13,5])
    plt.plot(series.index, series, 'o', color='g', linewidth=3)
    plt.plot(forecast.index, forecast.pred, '-', color=(1,0,0,0.9), linewidth=3)
    if PLOT_RAW:
        plt.plot(forecast.index, forecast.pred_raw, '.', color='r')
    plt.fill_between(forecast.index, forecast.ci_lower, forecast.ci_upper, color=(1,0,0,0.1))
    if y_lims:
        plt.ylim(y_lims)
    plt.xticks(rotation=45, fontsize='small')
    plt.title(title)
    plt.legend(['True', 'Prediction'])
    plt.grid(True, 'both')
    plt.tight_layout()

### Italy data - Wrangling

In [3]:
df_italy_regions = pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv")
df_italy_regions.head()

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN


In [57]:
df_italy_regions.columns

Index(['data', 'stato', 'codice_regione', 'denominazione_regione', 'lat',
       'long', 'ricoverati_con_sintomi', 'terapia_intensiva',
       'totale_ospedalizzati', 'isolamento_domiciliare', 'totale_positivi',
       'variazione_totale_positivi', 'nuovi_positivi', 'dimessi_guariti',
       'deceduti', 'totale_casi', 'tamponi', 'casi_testati', 'note_it',
       'note_en'],
      dtype='object')

In [74]:
info_regions = df_italy_regions[['data', 'codice_regione', 'denominazione_regione', 'lat',
       'long', 'ricoverati_con_sintomi', 'terapia_intensiva',
       'totale_ospedalizzati', 'isolamento_domiciliare', 'totale_positivi',
       'variazione_totale_positivi', 'nuovi_positivi', 'dimessi_guariti',
       'deceduti', 'totale_casi', 'tamponi', 'casi_testati']]

# transaltion in English
info_regions.columns = ['date', 'region_code', 'region', 'lat',
       'long', 'hospitalised_w_symptoms', 'intensive_care',
       'tot_hospitalised', 'isolation', 'confirmed_cases',
       'var_confirmed_cases', 'new_confirmed_cases', 'recovered',
       'fatalities', 'tot_cases', 'tests', 'tested_cases']
info_regions.head()

,date,region_code,region,lat,long,hospitalised_w_symptoms,intensive_care,tot_hospitalised,isolation,confirmed_cases,var_confirmed_cases,new_confirmed_cases,recovered,fatalities,tot_cases,tests,tested_cases
0,2020-02-24T18:00:00,13,Abruzzo,42.351222,13.398438,0,0,0,0,0,0,0,0,0,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,40.639471,15.805148,0,0,0,0,0,0,0,0,0,0,0,NaN
2,2020-02-24T18:00:00,21,P.A. Bolzano,46.499335,11.356624,0,0,0,0,0,0,0,0,0,0,1,NaN
3,2020-02-24T18:00:00,18,Calabria,38.905976,16.594402,0,0,0,0,0,0,0,0,0,0,1,NaN
4,2020-02-24T18:00:00,15,Campania,40.839566,14.250850,0,0,0,0,0,0,0,0,0,0,10,NaN


## All the regions in Italy 

In [70]:
regions = ['Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Emilia-Romagna',
       'Friuli Venezia Giulia', 'Lazio', 'Liguria', 'Lombardia', 'Marche',
       'Molise', 'P.A. Bolzano', 'P.A. Trento', 'Piemonte', 'Puglia',
       'Sardegna', 'Sicilia', 'Toscana', 'Umbria', "Valle d'Aosta",
       'Veneto']

In [71]:
key = 'fatalities'
winsize = 6
save_png = False
kf_type = 'ensemble'
kf_params = {'kf_p':1, 'kf_r':100, 'kf_q':0.01, 'kf_n':1000}

In [72]:
def show_one_region(region, variable_of_interest): 
    key = variable_of_interest
    df_country = info_regions[info_regions.region == region].groupby('date').sum()

    df_country.index = pd.to_datetime(df_country.index, format='%Y-%m-%d').strftime('%m-%d')
    df_country = df_country[df_country.index >= '02-10'] # most measurement data starts from March
    y_lims = [min(df_country[key]) * 0.5, max(df_country[key]) * 1.5]
    test_plot(df_country[key], winsize, kf_type, kf_params, '%s: %d-day prediction of %s' % (region, winsize, key), y_lims=y_lims)
    if save_png:
        plt.savefig('%s_%s.png' % (region, key), dpi=144, transparent=False)
        plt.close()
    return 

In [73]:
@interact
def show_options(variable=['fatalities', 'confirmed_cases'], 
                            region=regions):
    return show_one_region(region, variable)

interactive(children=(Dropdown(description='variable', options=('fatalities', 'confirmed_cases'), value='fatal…

## Part II 
# Risk Index Creation

## Additional info to merge 


we need to add: 
1. Oxford stringency Index 
2. Population: density, population, area (north, centre, south)
3. Age distribution 
4. Vulnerable population 

### Oxford Stringency

In [223]:
## it happens that sometime Oxford stringency is leaving the latest index as "." therefore for us are nan values. 
## To compute the risk index we need to have at least a stringency value so the idea is to fill in the dots with 
# the latest value available. 

def last_available_stringency(stringency): 
    if np.isnan(stringency.unique()[-1]): 
        return stringency.unique()[-2]
    else:
        return stringency.unique()[-1]

In [222]:
oxford = pd.read_csv("https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/index_stringency.csv")
oxford.rename(columns = {"Unnamed: 0": "country", "Unnamed: 1": "country_code"}, inplace = True)
stringency_italy = oxford[oxford["country_code"]=="ITA"]
string_italy = stringency_italy.iloc[:, 2:].transpose()
string_italy.columns = ["italy_stringency"]
string_italy.index = pd.to_datetime(string_italy.index, format='%d%b%Y').strftime('%m-%d')
string_italy.replace(".", np.nan, inplace = True)
string_italy["italy_stringency"] = string_italy["italy_stringency"].astype(float)
last_stringency = last_available_stringency(string_italy["italy_stringency"])
string_italy.head()
display(print(f"last stringency available: {last_stringency}"))


last stringency available: 38.88999939


None

### Population: density, pop, area, etc

In [148]:
pop = pd.read_csv("Italy_region_pop_data.csv")
pop.head()

,regione,pop_19,density,Num_province,Geo
0,Abruzzo,1311580,121,4,South
1,Basilicata,562869,56,2,South
2,Calabria,1947131,128,5,South
3,Campania,5801692,424,5,South
4,Emilia-Romagna,4459477,199,9,North


In [251]:
pop['regione'].unique()

array(['Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Emilia-Romagna',
       'Friuli Venezia Giulia', 'Lazio', 'Liguria', 'Lombardia', 'Marche',
       'Molise', 'Piemonte', 'Puglia', 'Sardegna', 'Sicilia', 'Toscana',
       'P.A. Trento', 'Umbria', "Valle d'Aosta", 'Veneto', 'P.A. Bolzano'],
      dtype=object)

### Age Distribution 

In [254]:
age = pd.read_excel("age_perc_distribution_by_region.xlsx")
age.head()

,Region,0-10,11-20,21-30,31-40,41-50,51-60,61-70,71-80,81-110
0,Abruzzo,0.088946,0.089776,0.104107,0.121518,0.152129,0.152444,0.126688,0.094533,0.069858
1,Basilicata,0.083291,0.095868,0.114311,0.120584,0.147191,0.154706,0.127676,0.087193,0.069181
2,Calabria,0.094159,0.099397,0.118801,0.127879,0.144234,0.147290,0.123101,0.085146,0.059993
3,Campania,0.101884,0.112055,0.124274,0.127805,0.150087,0.147074,0.112944,0.078243,0.045633
4,Emilia-Romagna,0.094260,0.090313,0.094784,0.117850,0.161795,0.151502,0.118989,0.099136,0.071370


In [253]:
age['Region'].unique()

array(['Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Emilia-Romagna',
       'Friuli-Venezia Giulia', 'Lazio', 'Liguria', 'Lombardia', 'Marche',
       'Molise', 'Piemonte', 'P.A. Bolzano', 'P.A. Trento', 'Puglia',
       'Sardegna', 'Sicilia', 'Toscana', 'Umbria', "Valle d'Aosta",
       'Veneto'], dtype=object)

### Vulnerable Population 

In [258]:
vulnerable = pd.read_csv("Chronical_Disease_by_Region.csv")
vulnerable.rename(columns = {"Row Labels": "region"}, inplace = True)
vulnerable.head()

,region,chronically_ill_arthritis,chronically_ill_bronchitis,chronically_ill_diabetes,chronically_ill_nervous_disorder,chronically_ill_Hypertention,chronically_ill_allergy,chronically_ill_cardiac_disease,chronically_ill_osteoporosis,chronically_ill_ulcer,at_least_one_chronical_disease,at_least_two_chronical_disease,good_health_with_chronic_disease,good_health
0,Abruzzo,17.9,6.7,7.5,5.7,18.5,11.9,3.7,9.6,3.1,40.2,22.7,38.8,66.4
1,Basilicata,19.8,7.1,7.0,4.4,20.3,13.6,4.1,9.7,4.1,44.4,23.9,37.9,65.4
2,Calabria,19.4,6.4,8.2,5.2,18.6,10.3,4.7,10.6,3.8,39.5,24.3,30.3,62.9
3,Campania,15.4,5.8,6.5,4.7,19.3,11.4,3.5,8.3,1.9,37.5,21.4,38.6,69.5
4,Emilia-Romagna,17.2,5.8,5.3,3.9,17.2,11.1,4.6,6.8,2.9,43.3,20.9,43.8,69.1


In [260]:
vulnerable['region'].unique()

array(['Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Emilia-Romagna',
       'Friuli Venezia Giulia', 'Lazio', 'Liguria', 'Lombardia', 'Marche',
       'Molise', 'Piemonte', 'P.A. Bolzano', 'P.A. Trento', 'Puglia',
       'Sardegna', 'Sicilia', 'Toscana', 'Umbria', "Valle d'Aosta",
       'Veneto'], dtype=object)

## Risk Index Creation  - TO discuss with Sarah

In [279]:
from math import exp

## TO DISCUSS WITH THE TEAM
def risk_index(cases_acceleration, npi_stringency_index, pop61_70, pop71_80, pop81_110, CI_bronchitis, CI_diabetes, CI_hypertension, **kwargs):
    elder_pop= pop61_70 + pop71_80 + pop81_110
    chronically_ill = (CI_bronchitis + CI_diabetes + CI_hypertension)/100
    
    if np.isnan(npi_stringency_index):
        raw_index =  (0.25 * cases_acceleration + 
                      0.25 * (1/last_stringency) + 
                      0.25 * elder_pop +
                      0.25 * chronically_ill)
    else:
        raw_index =  (0.25 * cases_acceleration + 
                      0.25 * (1/npi_stringency_index) + 
                      0.25 * elder_pop +
                      0.25 * chronically_ill)
    if raw_index < 0:
        return 1 - 1/(1 + exp(raw_index))
    else:
        return 1/(1 + exp(-raw_index))


## TO DISCUSS WITH THE TEAM
def discrete_risk_idx(risk_index):
    if risk_index < 0.3:
        return "low risk"
    elif risk_index < 0.5:
        return 'moderate risk'
    elif np.isnan(risk_index): 
        return np.nan
    else:
        return "high risk"


# Merge all the datasets 


## Create a for loop to compute the whole regions

In [235]:
all_regions = pd.DataFrame()

In [280]:
%%time 

for region in regions: 
    print(f"{region}")
    df_region = info_regions[info_regions.region == region].groupby('date').sum()
    # print(len(df_region))
    df_region['region'] = region

    df_region.index = pd.to_datetime(df_region.index, format='%Y-%m-%d').strftime('%m-%d')
    df_region = df_region[df_region.index >= '02-10'] # most measurement data starts from March
    
    dday = str(df_region.index[-1])
    
    result = kalman_test(df_region["confirmed_cases"], winsize, kf_type, kf_params)
    forecast = kalman_forecast(df_region["confirmed_cases"], winsize, kf_type, kf_params)
    print ("   applying Kalman Filter")
    
    df_region.reset_index(inplace = True)
    df_region.rename(columns = {"index": 'date'}, inplace = True)
    result.reset_index(inplace = True)
    result.rename(columns = {"index": 'date'}, inplace = True)
    forecast.reset_index(inplace = True)
    forecast.rename(columns = {"index": 'date'}, inplace = True)
    
    #now you have both predictions of the past (and actuals) and the forecast for the next 6 days 
    predictions = pd.concat([result, forecast])
    
    region_pred = df_region.merge(predictions, on="date", how = 'outer')
    print ("   merging predictions")
    
    ## fixing some detlias after the merge 
    recurrent_var = ["region_code", "lat", "long", "region"]
    for var in recurrent_var:
        region_pred[var].fillna(region_pred[var][0], inplace=True)
    
    ## calculate slope                                 <-- TO DISCUSS WITH THE TEAM 
    region_pred['diff_pred'] = region_pred['pred'].diff()
    region_pred['infection_gradient_1'] = np.gradient(region_pred['pred'].rolling(center=False,window=1).mean())
    region_pred['infection_gradient_2'] = np.gradient(region_pred['pred'].rolling(center=False,window=2).mean())
    print ("   calculating slope")
    
    ## save each file just in case 
    region_pred.to_csv(f"./each_region/{region}_pred_{dday}.csv")
    print ("   saving file")
    
    ## merge the external dataset together 
    whole_region = region_pred.merge(string_italy, left_on="date", right_on = string_italy.index)
    whole_region = whole_region.merge(pop, left_on = 'region', right_on = 'regione')
    whole_region = whole_region.merge(age, left_on = 'regione', right_on = 'Region')
    whole_region = whole_region.merge(vulnerable, left_on = 'Region', right_on = 'region')
    print ("   merging external files: oxford, population, age, vulnerable")

    ## compute the risk index 
    whole_region['risk_index'] = whole_region[['infection_gradient_1', 
                                               'italy_stringency', 
                                               '61-70', 
                                               '71-80', 
                                               '81-110', 
                                               'chronically_ill_bronchitis', 
                                               'chronically_ill_diabetes', 
                                               'chronically_ill_Hypertention']].apply(lambda x: risk_index(x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7]), axis =1)
    whole_region['risk_index_disc'] = whole_region['risk_index'].apply(lambda x: discrete_risk_idx(x))
    ####### you have a full region now!!! 
    print ("   calculating risk index")
    
    ## let's add all up -
    all_regions = all_regions.append(whole_region, ignore_index = True)
    
    print("   completed")
    
    # on to the next region! 

Abruzzo
   applying Kalman Filter
   merging predictions
   calculating slope
   saving file
   merging external files: oxford, population, age, vulnerable
   calculating risk index
   completed
Basilicata


/Users/erikaagostinelli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Users/erikaagostinelli/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


   applying Kalman Filter
   merging predictions
   calculating slope
   saving file
   merging external files: oxford, population, age, vulnerable
   calculating risk index
   completed
Calabria
   applying Kalman Filter
   merging predictions
   calculating slope
   saving file
   merging external files: oxford, population, age, vulnerable
   calculating risk index
   completed
Campania
   applying Kalman Filter
   merging predictions
   calculating slope
   saving file
   merging external files: oxford, population, age, vulnerable
   calculating risk index
   completed
Emilia-Romagna
   applying Kalman Filter
   merging predictions
   calculating slope
   saving file
   merging external files: oxford, population, age, vulnerable
   calculating risk index
   completed
Friuli Venezia Giulia
   applying Kalman Filter
   merging predictions
   calculating slope
   saving file
   merging external files: oxford, population, age, vulnerable
   calculating risk index
   completed
Lazio
   a

In [282]:
all_regions.to_csv("IT_all_regions_COMPLETE.csv")

In [283]:
all_regions.head()

,0-10,11-20,21-30,31-40,41-50,51-60,61-70,71-80,81-110,Geo,Num_province,Region,at_least_one_chronical_disease,at_least_two_chronical_disease,chronically_ill_Hypertention,chronically_ill_allergy,chronically_ill_arthritis,chronically_ill_bronchitis,chronically_ill_cardiac_disease,chronically_ill_diabetes,chronically_ill_nervous_disorder,chronically_ill_osteoporosis,chronically_ill_ulcer,ci_lower,ci_upper,confirmed_cases,date,density,diff_pred,fatalities,good_health,good_health_with_chronic_disease,history,hospitalised_w_symptoms,infection_gradient_1,infection_gradient_2,infections_var,intensive_care,isolation,italy_stringency,lat,long,new_confirmed_cases,obs,pop_19,pred,pred_raw,recovered,region_code,region_x,region_y,regione,risk_index,risk_index_disc,tested_cases,tests,tot_cases,tot_hospitalised,var_confirmed_cases
0,0.088946,0.089776,0.104107,0.121518,0.152129,0.152444,0.126688,0.094533,0.069858,South,4,Abruzzo,40.2,22.7,18.5,11.9,17.9,6.7,3.7,7.5,5.7,9.6,3.1,NaN,NaN,0.0,02-24,121,NaN,0.0,66.4,38.8,NaN,0.0,NaN,NaN,NaN,0.0,0.0,69.910004,42.351222,13.398438,0.0,NaN,1311580,NaN,NaN,0.0,13.0,Abruzzo,Abruzzo,Abruzzo,NaN,NaN,0.0,5.0,0.0,0.0,0.0
1,0.088946,0.089776,0.104107,0.121518,0.152129,0.152444,0.126688,0.094533,0.069858,South,4,Abruzzo,40.2,22.7,18.5,11.9,17.9,6.7,3.7,7.5,5.7,9.6,3.1,NaN,NaN,0.0,02-25,121,NaN,0.0,66.4,38.8,NaN,0.0,NaN,NaN,NaN,0.0,0.0,69.910004,42.351222,13.398438,0.0,NaN,1311580,NaN,NaN,0.0,13.0,Abruzzo,Abruzzo,Abruzzo,NaN,NaN,0.0,5.0,0.0,0.0,0.0
2,0.088946,0.089776,0.104107,0.121518,0.152129,0.152444,0.126688,0.094533,0.069858,South,4,Abruzzo,40.2,22.7,18.5,11.9,17.9,6.7,3.7,7.5,5.7,9.6,3.1,NaN,NaN,0.0,02-26,121,NaN,0.0,66.4,38.8,NaN,0.0,NaN,NaN,NaN,0.0,0.0,69.910004,42.351222,13.398438,0.0,NaN,1311580,NaN,NaN,0.0,13.0,Abruzzo,Abruzzo,Abruzzo,NaN,NaN,0.0,13.0,0.0,0.0,0.0
3,0.088946,0.089776,0.104107,0.121518,0.152129,0.152444,0.126688,0.094533,0.069858,South,4,Abruzzo,40.2,22.7,18.5,11.9,17.9,6.7,3.7,7.5,5.7,9.6,3.1,NaN,NaN,1.0,02-27,121,NaN,0.0,66.4,38.8,NaN,1.0,NaN,NaN,NaN,0.0,0.0,69.910004,42.351222,13.398438,1.0,NaN,1311580,NaN,NaN,0.0,13.0,Abruzzo,Abruzzo,Abruzzo,NaN,NaN,0.0,33.0,1.0,1.0,1.0
4,0.088946,0.089776,0.104107,0.121518,0.152129,0.152444,0.126688,0.094533,0.069858,South,4,Abruzzo,40.2,22.7,18.5,11.9,17.9,6.7,3.7,7.5,5.7,9.6,3.1,NaN,NaN,1.0,02-28,121,NaN,0.0,66.4,38.8,NaN,1.0,NaN,NaN,NaN,0.0,0.0,69.910004,42.351222,13.398438,0.0,NaN,1311580,NaN,NaN,0.0,13.0,Abruzzo,Abruzzo,Abruzzo,NaN,NaN,0.0,33.0,1.0,1.0,0.0
